### Spark Cluster Performance
The purpose of this notebook is to demonstrate the advantages of a Spark cluster query as opposed to a Postgres database query. It is best to complete the entire notebook in one sitting, as opposed to leaving it and coming
back later. If you choose to leave it and come back to it later, you most likely experience glitches in the server.
It is also important to run each of these top to bottom, and WAIT until one cell is finished before starting the next.

#### Database Structure
* reviews

|Field |	Type |	Allow Null |	Default Value| 
|------|-------|-------|------|
|product_id |	varchar(20) |	Yes	| x |
|price |	float4	|Yes | x  |	
|user_id |	varchar(50) |	Yes	| x |
|profile_name |	varchar(100) |	Yes	 | x |
|time |	timestamp(6) WITH TIME ZONE |	Yes |x |	
|score |	float4 |	Yes |x	 |
|title |	text |	Yes | x |	
|summary |	text |	Yes	  |  x|
|text | 	text |	Yes	 |x  |
|helpfulness_score |	float4 |	Yes  |x  |	
|helpfulness_count |	float4 |	Yes	 |x |
|counter_id	 | int4	| No |	nextval('reviews_counter_id_seq'::regclass) |

#### Accessing Spark
Creating the SQL context from the spark context, which is the entry point into the relational functionality of our spark cluster.  Importing libraries. We need the time library to use the time.time() function.

**In each fo the cells below, make sure you wait for the [\*] to turn into a number before hitting the next cell. This code does take a time to come back. **

In [2]:
import time
from pyspark import SQLContext, SparkContext, SparkConf
sqlContext = SQLContext(sc)


The code below creates the DataFrame from the SQLContext created above.  The DataFrame is based on the contents of our JSON file. 

This file contains the content from the **Amazon Product Reviews Dataset**. DataFrames allow us to manipulate and interact with structured data via a domain-specific language. Our domain-specific language is Python. 

The code below creates the DataFrame from the SQLContext based on the contents of our JSON file. This file contains the content from the reviews database. DataFrames allow us to manipulate and interact with structured data via a domain-specific language. Our domain-specific language is Python. 

In [3]:
## EXCEPT, note that this cell will not come back and often leaves the [*] in the execution notation to the left. 
df = sqlContext.read.json("In/reviews.json")

In [4]:
# This prints the schema. 

df.printSchema()

root
 |-- helpfuless_count: long (nullable = true)
 |-- helpfuless_score: long (nullable = true)
 |-- price: string (nullable = true)
 |-- productId: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- score: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- text: string (nullable = true)
 |-- time: string (nullable = true)
 |-- title: string (nullable = true)
 |-- userId: string (nullable = true)

In [5]:
# This is a basic query that selects twenty scores (a.k.a. ratings) of different movies and TV shows. The 'twenty'
# is a constraint on the system to prevent the accidental retrieval of about thirty-four million rows. 
# This query does not specify how many to return.

df2 = df.select("score").show()

+-----+
|score|
+-----+
|  4.0|
|  5.0|
|  3.0|
|  4.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  4.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
+-----+
only showing top 20 rows

In [6]:
# You can register a DataFrame as a table for the purpose of running an SQL query on the dataset. 

sqlContext.registerDataFrameAsTable(df, "reviews")
df3 = sqlContext.sql("SELECT productId, price from reviews limit 5")
df3.collect()

[Row(productId=u' B000179R3I', price=u' unknown'), Row(productId=u' B000GKXY34', price=u' 17.99'), Row(productId=u' B000GKXY34', price=u' 17.99'), Row(productId=u' 1882931173', price=u' unknown'), Row(productId=u' B00002066I', price=u' 15.99')]

In [10]:
# Here we will run a more time-consuming query since our dataset is gigantic. (When I ran this, it took almost 
# five minutes, and my internet connection is fairly good.)

start = time.time()
df4 = sqlContext.sql("SELECT COUNT(DISTINCT title) FROM reviews")
df4.collect()
end = time.time()
print(end - start)

285.009870052

In [ ]:
# You should see a large difference between this time and the runtime of the postgres query. (Mine was about half
# the time of the postgres query.)

In [11]:
## Look at scores great than 4 
start = time.time()
df4 = df.filter(df.score>4).show()
#df4.collect()
end = time.time()
print(end - start)
df.printSchema()

+----------------+----------------+--------+-----------+--------------------+-----+--------------------+--------------------+-----------+--------------------+---------------+
|helpfuless_count|helpfuless_score|   price|  productId|         profileName|score|             summary|                text|       time|               title|         userId|
+----------------+----------------+--------+-----------+--------------------+-----+--------------------+--------------------+-----------+--------------------+---------------+
|               0|               0|   17.99| B000GKXY34|          M. Gingras|  5.0|          Great fun!| Got these last C...| 1262304000| Nun Chuck, Novel...|  ADX8VLDUOL7BG|
|               4|               3|   15.99| B00002066I|             unknown|  5.0|           Inspiring| I hope a lot of ...|  939772800|                  ah|        unknown|
|               0|               0|   15.99| B00002066I|   Stephen McClaning|  5.0|            Great CD| My lovely Pat ha...|

In [12]:
#Lets look at helpfulness scores greater than 40
from pyspark.sql.functions import col 

numeric_filtered = df.where(
    (col('helpfuless_score')  > 40))
numeric_filtered.show()

+----------------+----------------+--------+-----------+--------------------+-----+--------------------+--------------------+-----------+--------------------+---------------+
|helpfuless_count|helpfuless_score|   price|  productId|         profileName|score|             summary|                text|       time|               title|         userId|
+----------------+----------------+--------+-----------+--------------------+-----+--------------------+--------------------+-----------+--------------------+---------------+
|              81|              74|   25.97| 0802841899| Christopher J. Bray|  5.0| Ecclesiological ...| With the publica...|  955411200| The Church of Ch...|  ARI272XF8TOL4|
|             170|             169| unknown| B0000630MQ|     David Lindamood|  5.0| Great Compact Ch...| I need a fast, c...| 1020124800| Kodak Max K2000 ...| A1IT7KPZW9DY05|
|              46|              45| unknown| B0000630MQ|     Fred "mrdata22"|  5.0| A simple, reliab...| After reading th...|

In [13]:
#Lets look at helpfulness scores greater than 40 and a score greater than 4
from pyspark.sql.functions import col 

numeric_filtered = df.where(
    (col('helpfuless_score')  > 40))
numeric_filtered.show()

numeric_filtered2 = df.where(
    (col('score')  > 4))
numeric_filtered2.show()

# I print both out so that you can see the first group has the helpfulness score greater than 40 and the second group 
# meets that condition AND has a score greater than 4. 

+----------------+----------------+--------+-----------+--------------------+-----+--------------------+--------------------+-----------+--------------------+---------------+
|helpfuless_count|helpfuless_score|   price|  productId|         profileName|score|             summary|                text|       time|               title|         userId|
+----------------+----------------+--------+-----------+--------------------+-----+--------------------+--------------------+-----------+--------------------+---------------+
|              81|              74|   25.97| 0802841899| Christopher J. Bray|  5.0| Ecclesiological ...| With the publica...|  955411200| The Church of Ch...|  ARI272XF8TOL4|
|             170|             169| unknown| B0000630MQ|     David Lindamood|  5.0| Great Compact Ch...| I need a fast, c...| 1020124800| Kodak Max K2000 ...| A1IT7KPZW9DY05|
|              46|              45| unknown| B0000630MQ|     Fred "mrdata22"|  5.0| A simple, reliab...| After reading th...|

## Imagine Your Own Queries of the Amazon Reviews Database
What are some queries you might imagine for Amazon product reviews?
 - Is there a product title you would like to know the average rating for?
 - What ratings levels are the most useful? High or low?

In [22]:
# This is an example of a query used to find the average rating of reviews containing the product title "Amazon.com".
df4 = sqlContext.sql("select avg(score) from reviews where title like 'Amazon.com'")
df4.collect()

# This is an example of how you could determine whether high rating levels are more useful, or low ones. You could 
# query for the highest 100 ratings, and the lowest 100 ratings and read the reviews. 

df4 = sqlContext.sql("select score,title,summary from reviews group by score order by score desc limit 100")
df4.collect()
df5 = sqlContext.sql("select score,title,summary from reviews group by score order by score asc limit 100")
df5.collect()

u"expression 'title' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;"
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/context.py", line 580, in sql
    return DataFrame(self._ssql_ctx.sql(sqlQuery), self)
  File "/usr/lib/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 813, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 51, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u"expression 'title' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;"

